## Import library

In [1]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import utils 
from tempfile import TemporaryFile

# save numpy array as npy file
from numpy import asarray
from numpy import save

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
[MLENS] backend: threading


## Load data

In [3]:
treatment = 't'
outcome = 'yf'
col =  ["t", "yf", "ycf", "mu0", "mu1" ]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["t"]

RESULT_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/result"

In [5]:
PATH_TRAIN = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.train.npz"
PATH_TEST = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/ihdp_npci_1-100.test.npz"

rmse_incre_seq  = []
for index_ in tqdm(range(1,101)):    
    data = utils.load_data(PATH_TRAIN, index_)
    data_test = utils.load_data(PATH_TEST, index_)

    #Incremental propensity score
    ## Fit treatment
    model_t = LogisticRegression()
    model_t.fit(data[cov], data[treatment])
    

    ## Fit outcome
    model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
    model_y.fit(data[features], data[outcome])

    delta = 32.99
    influence = ipse.influence_function(data_test, treatment, cov, outcome, features, delta, model_y, model_t)
    true_effect = data_test['mu1'] - data_test['mu0']
    
    rmse_incre = utils.rmse_ite(influence, true_effect)
    rmse_incre_seq.append(rmse_incre)
    
    print("Data index {}. RMSE {:.2f}".format(index_, rmse_incre))

  1%|          | 1/100 [00:11<18:59, 11.51s/it]

Data index 1. RMSE 6.85


  2%|▏         | 2/100 [00:17<16:14,  9.95s/it]

Data index 2. RMSE 3.26


  3%|▎         | 3/100 [00:23<14:01,  8.67s/it]

...

Data index 98. RMSE 7.13


 99%|█████████▉| 99/100 [13:28<00:09,  9.21s/it]

Data index 99. RMSE 2.79


100%|██████████| 100/100 [13:33<00:00,  8.14s/it]

Data index 100. RMSE 2.79


In [7]:
np.mean(rmse_incre)